In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.description'):
    shutil.rmtree('./file/4.description')

os.makedirs('./file/4.description')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.item/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(input_.loc[a, 'Description Url'],
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
            
                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
            
                # = = = = = = = = = = = = = = =
            
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
            
                list_div = html.xpath('//div')

                # = = = = = = = = = = = = = = =

                fitment = ''
                for div in list_div:
                    if div.xpath('./h4/text()') and div.xpath('./h4/text()')[0].strip() == 'FITMENT':
                        list_text = [text.strip() for text in div.xpath('./following-sibling::div[1]/li/text()')]
                
                while '' in list_text:
                    list_text.remove('')
                
                fitment = '\n'.join(list_text)

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Description Url': input_.loc[a, 'Description Url'],
                                         'Fitment': fitment}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')


            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()
    
            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Item Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.description/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-description_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.description'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.description/{file}',
              f'''./file/4.description/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：81

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 1.144647596897
[剩余数量：51] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 7.362770712932
[剩余数量：50] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 12.362666165897
[剩余数量：49] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 8.143495529987
[剩余数量：48] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 28.382827006251
[剩余数量：47] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 11.142289967477
[剩余数量：46] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 26.383199849535
[剩余数量：45] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 16.143369543179
[剩余数量：44] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 22.362782882166
[剩余数量：43] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 19.382726430799
[剩余数量：42] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 25.385419766159
[剩余数量：41] - [当前时间：10:53:21]

[状态：ok，尝试次数：1] - 100.00% - crawler_1 > 17.362921110522
[剩余数量：40] - [当前时

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2050.00it/s]

Done ~
